# IMPORT DATA, PACKAGES, AND FUNCTIONS

## import the relevant functions and packages

In [1]:
import pandas as pd
import numpy as np
from test_case_generator_functions import alphabet

# TEST CASE TYPE REFERENCE INFORMATION

In [2]:
uid = 'UID-243' 
theme = 'Names where name parts are Modified'
category = 'Transpositions'
sub_category = '> 2 transpositions (different name part)'
entity_type = 'Entity'

## download the OFAC list from the web

In [3]:
ofac_list_download = pd.read_csv('https://www.treasury.gov/ofac/downloads/sdn.csv', header=None)

In [4]:
ofac_list = ofac_list_download[[0,1,2]]
ofac_list.columns = ['uid', 'name', 'entity_type']

# FILTER FOR THE REQUIREMENTS OF THE TEST CASE TYPE

## filter for the requirements of the specific test case type

In [5]:
ofac_list_filtered = ofac_list[(ofac_list.entity_type == '-0- ')] # only evaluate entities

## randomly choose 10 rows

In [6]:
while True:
    ofac_list_sampled = ofac_list_filtered.sample(n = 10)
    cnt = [0] * 10
    for ind, name in enumerate(ofac_list_sampled['name']):
        split_name = name.upper().split(' ')
        
        for i in split_name:
            if all(j not in alphabet() for j in i):
                split_name.remove(i) # remove word with all numbers
            elif len(i) == 1:
                split_name.remove(i) # remove word with only 1 letter
                
        cnt[ind] += len(split_name)

    if all(i > 2 for i in cnt):
        break

ofac_list_sampled

,uid,name,entity_type
904,8372,I.P.C. MARKETING LIMITED,-0-
1592,10644,"CAZPER IMPORTACIONES, S.A. DE C.V.",-0-
8254,30627,CENTRAL COMMISSION OF THE REPUBLIC OF BELARUS ...,-0-
7209,27166,SHAFAGH SENOBAR YAZD COMPANY LIMITED,-0-
4452,19790,MINISTRY OF PEOPLE'S SECURITY CORRECTIONAL BUREAU,-0-
7994,29866,ELLA TOURISM COMPANY,-0-
6565,25841,PALM SERVICE LIMITED,-0-
6869,26509,YAROSLAVSKY SHIPBUILDING PLANT,-0-
7674,28658,ARMED FORCES SOCIAL SECURITY INVESTMENT COMPANY,-0-
2717,13440,SYRIAN ARAB AIRLINES,-0-


# CREATE THE TEST CASES

## create blank final test cases table

In [7]:
final_test_cases = pd.DataFrame(columns=['UID', 'Theme','Category','Sub-category','Entity-Type','Test Case ID' , 'OFAC List UID', 'Original Name','Test Case Name'])
final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name


## run loop to generate the test cases

In [8]:
for index, row in ofac_list_sampled.iterrows():
    original_name = row['name'].upper()
    split_original_name = original_name.split(' ') # split name into word
    final_test_name = split_original_name.copy()
    
    for i in split_original_name:
        if all(j not in alphabet() for j in i):
            split_original_name.remove(i) # remove word with all numbers
        elif len(i) == 1:
            split_original_name.remove(i) # remove word with only 1 letter
    
    if len(split_original_name) == 3:
        loop_time = 3
    else:
        loop_time = np.random.randint(3, len(split_original_name))
        
    for i in range(loop_time):
        replace_word = np.random.choice(split_original_name) # randomly choose word to be transposed
        split_original_name.remove(replace_word) # remove chosen word
        
        random_list = list(range(len(replace_word))) # randomly choose letter to be transposed
        while True:
            random_letter_index = np.random.choice(random_list)
            random_letter = replace_word[random_letter_index]
            if random_letter in alphabet():
                break
        random_list.remove(random_letter_index) # remove chosen letter index
        
        random_place = np.random.choice(random_list) # randomly choose place to transpose to
        
        replace_word_list = list(replace_word)
        del replace_word_list[random_letter_index] # remove chosen letter
        replace_word_list.insert(random_place, random_letter) # add transposed letter
        
        final_test_name[final_test_name.index(replace_word)] = ''.join(replace_word_list)

    final_test_name = ' '.join(final_test_name)
    
    final_test_cases.loc[len(final_test_cases)] = [uid, theme, category, sub_category, entity_type, uid + ' - ' + str(index), row['uid'], row['name'], final_test_name] # append to the dataframe

final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name
0,UID-243,Names where name parts are Modified,Transpositions,> 2 transpositions (different name part),Entity,UID-243 - 904,8372,I.P.C. MARKETING LIMITED,.IP.C. ARKETIMNG LIMEITD
1,UID-243,Names where name parts are Modified,Transpositions,> 2 transpositions (different name part),Entity,UID-243 - 1592,10644,"CAZPER IMPORTACIONES, S.A. DE C.V.","CAZPER IMPORTAICONES, .A.S DE .V.C"
2,UID-243,Names where name parts are Modified,Transpositions,> 2 transpositions (different name part),Entity,UID-243 - 8254,30627,CENTRAL COMMISSION OF THE REPUBLIC OF BELARUS ...,CETNRAL COMMISSION FO ETH REPUBLCI OF BELRUSA ...
3,UID-243,Names where name parts are Modified,Transpositions,> 2 transpositions (different name part),Entity,UID-243 - 7209,27166,SHAFAGH SENOBAR YAZD COMPANY LIMITED,ASHFAGH SENOBAR YAZD CNOMPAY LIMTEID
4,UID-243,Names where name parts are Modified,Transpositions,> 2 transpositions (different name part),Entity,UID-243 - 4452,19790,MINISTRY OF PEOPLE'S SECURITY CORRECTIONAL BUREAU,MINSTRIY FO EOPLE'PS SCUERITY OCRRECTIONAL BUREAU
5,UID-243,Names where name parts are Modified,Transpositions,> 2 transpositions (different name part),Entity,UID-243 - 7994,29866,ELLA TOURISM COMPANY,EALL TOURSIM CMPANOY
6,UID-243,Names where name parts are Modified,Transpositions,> 2 transpositions (different name part),Entity,UID-243 - 6565,25841,PALM SERVICE LIMITED,PLAM SIERVCE LIITMED
7,UID-243,Names where name parts are Modified,Transpositions,> 2 transpositions (different name part),Entity,UID-243 - 6869,26509,YAROSLAVSKY SHIPBUILDING PLANT,YRAOSLAVSKY SHIIPBULDING LPANT
8,UID-243,Names where name parts are Modified,Transpositions,> 2 transpositions (different name part),Entity,UID-243 - 7674,28658,ARMED FORCES SOCIAL SECURITY INVESTMENT COMPANY,ARMED FOCESR SOCIAL SECURIYT INVESTMENT CONMPAY
9,UID-243,Names where name parts are Modified,Transpositions,> 2 transpositions (different name part),Entity,UID-243 - 2717,13440,SYRIAN ARAB AIRLINES,SYIANR RAAB ARILINES
